# Linear Optimization

In this notebook we will learn how to solve linear optimiztion problems in Python using the package `Pyomo` and the open source optimization solver `glpk`.

`Pyomo` allows us to formulate mathematical optimization problems conveniently in Python, and passes these problems to a solver backend. `Pyomo` can use many solvers: `glpk` is free and convenient for small and medium-sized optimization problems. 

The problem we will solve is from earlier this week, where a Home Appliance company is trying to maximize its profit by deciding how many washers and dryers to produce, subject to constraints on manufacturing, testing, and assembly resources. 

Recall that we defined $x_1$ as the number of washers produced per day, $x_2$ as the number of dryers produced per day. Washers and dryers produced a profit of \\$100 and \\$200 per unit, respectively. 

The company’s resources are such that only 20 hours of manufacturing, 20 hours of assembly and 25 hours of testing are available per day. Each washer has to spend 1 hr in manufacturing, 2 hrs in assembly and 2 hrs in testing. Each dryer has to spend 2 hrs in manufacturing, 1 hr in assembly and 2 hrs in testing.

We formulated this problem mathematically as follows: 

$$
\begin{aligned}
& \underset{x}{\text{maximize}}
& & 100 x_1 + 120 x_2\\
& \text{subject to}
& & x_1 + 2 x_2 \leq 20 \\
& & & 2x_1 +  x_2 \leq 20 \\
& & & 2x_1 + 2 x_2 \leq 25 \\
&&& x_1, x_1 \geq 0
\end{aligned}
$$

In [1]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

Next, we need to define our decision variables. We do this using the `Var` class, and save these variables as `model.x`. We first specify how this variable will be indexed: since we have $x_1$ and $x_2$, we define the variables with the index list `[1,2]`. You can define indices in `Pyomo` using many different approaches, including dictionaries and functions - for this relatively simple variable, a simple list will work well. We also specify what type of variables these are: in this case, they are real numbers greater than or equal to zero, so we specify `Domain` as `NonNegativeReals`. This takes care of the final constraint in the mathematical formulation. If we wanted to allow our variables to be negative, we could specify the `Domain` as `Reals`.

In [2]:
#declare decision variables. [1,2] defines the index set
model.x = Var([1,2],domain=NonNegativeReals)

In the next line, we define our objective function using the `Objective` class, and save it in our model object as model.profit. We give the expression `expr` as `100*model.x[1] + 120*model.x[2]`, just as in the mathematical formulation, and specify that this is a maximization problem by setting `sense` as `maximize`.

In [3]:
#declare objective
model.profit = Objective(expr = 100*model.x[1] + 120*model.x[2], sense=maximize)

Now, we can define our constraints - we instantiate one member of the `Constraint` class for each one of our mathematical constraints - except for the non-negativity constraints, which we specified as part of our variable definitions. We write the constraint expression for `expr` similarly to the objective function, but this time, we include the inequality `<=` to set the constraint. If we instead wanted an equality constraint, we would use `==`.

We then can get a summary of our model using the `.pprint()` method.

In [4]:
#declare constraints
model.manufacturing = Constraint(expr = model.x[1] +2*model.x[2] <= 20)
model.assembly = Constraint(expr = 2*model.x[1] + model.x[2] <= 20)
model.testing = Constraint(expr = 2*model.x[1] + 2*model.x[2] <= 25)

model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 100*x[1] + 120*x[2]

3 Constraint Declarations
    assembly : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 2*x[1] + x[2] :  20.0 :   True
    manufacturing : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : x[1] + 2*x[2] :  20.0 :   True
    testing : Size=1, Index=None, Active=True
        Key  : Lower : 

We can now solve our model. We use `SolverFactory` to specify the solver - in this case, `glpk`, call the method `.solve()` on our model to perform the optimization, then call `.write()` to get detailed solution output.

In [5]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1400.0
  Upper bound: 1400.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 7
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005564451217651367
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

The output summarizes the number of constraints and variables, and tells us that our solver successfully achieved optimality with `Termination condition: optimal`. 

In order to see our solution, we can see the objective function value with `model.value()`, view the optimal variable values with `model.x[1].value` and `model.x[2].value`. By running the code, we see that an optimal profit of \\$1400 is achieved with $x_1 = 5.0$ washers and $x_2 = 7.5$ dryers, just as seen in our earlier lesson.

In [6]:
#display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x1 = ', model.x[1].value)
print('x2 = ', model.x[2].value)


Profit =  1400.0

Decision Variables
x1 =  5.0
x2 =  7.5


Similarly, we can view the values of our constraint expressions at optimality using the code below. As seen in the earlier lesson, manufacturing and testing capcity are at their maximum allowed values, while assembly, at 17.5 hours, less than its maximum value of 20 hours, is an inactive constraint.

In [7]:
print('\nConstraints')
print('Manufacturing  = ', model.manufacturing())
print('Assembly = ', model.assembly())
print('Testing = ', model.testing())


Constraints
Manufacturing  =  20.0
Assembly =  17.5
Testing =  25.0


Suppose we wanted to consider investing in the expansion of our manufacturing and testing resources - the currently limiting constraints on our profit. Would this be worth it? 

To analyze this, we can modify our constraints by increasing their limits using the `.deactivate()` method shown below, and define new constraints with increased limits, adding one hour to each resource. We then re-solve the model.

In [8]:
#deactivate the current manufacturing and testing constraints
model.manufacturing.deactivate()
model.testing.deactivate()

#define new constraints with increased resource limits
model.manufacturing_new = Constraint(expr = model.x[1] +2*model.x[2] <= 21 )
model.testing_new = Constraint(expr = 2*model.x[1] +2*model.x[2] <= 26 )

#resolve model
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x1 = ', model.x[1].value)
print('x2 = ', model.x[2].value)

print('\nConstraints')
print('Manufacturing  = ', model.manufacturing_new())
print('Assembly = ', model.assembly())
print('Testing = ', model.testing_new())


Profit =  1460.0

Decision Variables
x1 =  5.0
x2 =  8.0

Constraints
Manufacturing  =  21.0
Assembly =  18.0
Testing =  26.0


Our profit has increased to \\$1460 - if we have reason to believe that adding an hour to both our manufacturing and testing resouces would cost less than  \\$60, this investment may be worth it. 

We have now solved a linear optimization in Pyomo. This was a relatively small problem, which we were able solve "by hand" in our earlier lesson. But using `Pyomo` and an appropriate backend solver, we can solve large models with thousands or even millions of variables.